In [1]:
#Import libraries
from pyhive import presto
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import h3
from shapely.geometry import Polygon, Point
import calendar
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [2]:
#Define presto credentials
presto_host = 'presto.processing.yoda.run'
presto_port = '80'
username = 'aditya.bhattar@rapido.bike'

#Create connection to presto host
connection = presto.connect(presto_host,presto_port,username = username)

In [3]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, captain_location_hex_8, captain_location_latitude,
captain_location_longitude, city_name, customer_location_hex_8, customer_location_latitude, 
pickup_location_hex_8, pickup_location_latitude, pickup_location_longitude, drop_location_hex_8, drop_location_latitude, drop_location_longitude,
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason
from orders.order_logs_snapshot
where yyyymmdd >= '20230206'
and yyyymmdd <= '20230210'
and city_name in ('Hyderabad')
and service_obj_service_name = 'Link'
"""

#Load data into pandas table
df_copy = pd.read_sql(q, connection)

KeyboardInterrupt: 

In [ ]:
#Create a copy of the df retrived from presto
df = df_copy.copy()
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason
0,63e575ca2f0d965b714cca00,0.628,8860a24809fffff,17.483631,78.316765,Hyderabad,8860a24b09fffff,17.480679,8860a24b09fffff,17.480523,78.378223,8860a24809fffff,17.483631,78.316765,78.378220,040802,04,0400,Link,04:00,5,20230210,dropped,None
1,63e57c3d446060795b9eceee,0.195,8860a2591dfffff,17.432058,78.445732,Hyderabad,8860a2591dfffff,17.432005,8860a2591dfffff,17.431942,78.446724,8860a25911fffff,17.431355,78.441867,78.447182,043533,04,0430,Link,04:59,5,20230210,customerCancelled,order cancelled before rider accepted
2,63e57f61c3c8103bdee23301,NaN,8860a259b9fffff,17.444122,78.387210,Hyderabad,8860a259bbfffff,17.441753,8860a259bbfffff,17.441742,78.382489,8860b196b9fffff,17.496080,78.419732,78.382538,044857,04,0445,Link,04:59,5,20230210,expired,None
3,63e57f7a446060795b9ed072,2.417,8860b5251dfffff,17.329424,78.565979,Hyderabad,8860b52ecdfffff,17.369417,8860b52ecdfffff,17.369176,78.573872,8860b5251dfffff,17.329424,78.565979,78.573624,044922,04,0445,Link,04:59,5,20230210,dropped,None
4,63e5800b446060795b9ed0be,1.249,8860a25903fffff,17.440149,78.450714,Hyderabad,8860a25939fffff,17.442627,8860a25939fffff,17.442596,78.445930,8860a25903fffff,17.440149,78.450714,78.445938,045147,04,0445,Link,04:59,5,20230210,dropped,None


In [ ]:
#Make columns for second and minute and date
df['second'] = df['hhmmss'].apply(lambda x: x[4:])
df['minute'] = df['hhmmss'].apply(lambda x: x[2:4])
df['date'] = df['yyyymmdd'].apply(lambda x: x[6:])
df['count'] = 1
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,second,minute,date,count
0,63e575ca2f0d965b714cca00,0.628,8860a24809fffff,17.483631,78.316765,Hyderabad,8860a24b09fffff,17.480679,8860a24b09fffff,17.480523,78.378223,8860a24809fffff,17.483631,78.316765,78.378220,040802,04,0400,Link,04:00,5,20230210,dropped,None,02,08,10,1
1,63e57c3d446060795b9eceee,0.195,8860a2591dfffff,17.432058,78.445732,Hyderabad,8860a2591dfffff,17.432005,8860a2591dfffff,17.431942,78.446724,8860a25911fffff,17.431355,78.441867,78.447182,043533,04,0430,Link,04:59,5,20230210,customerCancelled,order cancelled before rider accepted,33,35,10,1
2,63e57f61c3c8103bdee23301,NaN,8860a259b9fffff,17.444122,78.387210,Hyderabad,8860a259bbfffff,17.441753,8860a259bbfffff,17.441742,78.382489,8860b196b9fffff,17.496080,78.419732,78.382538,044857,04,0445,Link,04:59,5,20230210,expired,None,57,48,10,1
3,63e57f7a446060795b9ed072,2.417,8860b5251dfffff,17.329424,78.565979,Hyderabad,8860b52ecdfffff,17.369417,8860b52ecdfffff,17.369176,78.573872,8860b5251dfffff,17.329424,78.565979,78.573624,044922,04,0445,Link,04:59,5,20230210,dropped,None,22,49,10,1
4,63e5800b446060795b9ed0be,1.249,8860a25903fffff,17.440149,78.450714,Hyderabad,8860a25939fffff,17.442627,8860a25939fffff,17.442596,78.445930,8860a25903fffff,17.440149,78.450714,78.445938,045147,04,0445,Link,04:59,5,20230210,dropped,None,47,51,10,1


In [ ]:
#Load data from cluster table
q = """
select * from datasets.city_cluster_hex
where city in ('Hyderabad')
and resolution = 8
"""
#Load data into pandas table
df_cluster_copy = pd.read_sql(q, connection)

In [ ]:
cluster_mapping = df_cluster_copy.copy()
cluster_mapping = cluster_mapping.rename(columns = {'hex_id':'pickup_location_hex_8'})
df = df.merge(cluster_mapping[['pickup_location_hex_8', 'cluster']], on = 'pickup_location_hex_8', how = 'left')
cluster_mapping = cluster_mapping.rename(columns = {'pickup_location_hex_8':'drop_location_hex_8'})
df = df.rename(columns = {'cluster':'pickup_cluster'})
df = df.merge(cluster_mapping[['drop_location_hex_8', 'cluster']], on = 'drop_location_hex_8', how = 'left')
df = df.rename(columns = {'cluster':'drop_cluster'})
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,second,minute,date,count,pickup_cluster,drop_cluster
0,63e575ca2f0d965b714cca00,0.628,8860a24809fffff,17.483631,78.316765,Hyderabad,8860a24b09fffff,17.480679,8860a24b09fffff,17.480523,78.378223,8860a24809fffff,17.483631,78.316765,78.378220,040802,04,0400,Link,04:00,5,20230210,dropped,None,02,08,10,1,Kothaguda 2,Lingampalli
1,63e57c3d446060795b9eceee,0.195,8860a2591dfffff,17.432058,78.445732,Hyderabad,8860a2591dfffff,17.432005,8860a2591dfffff,17.431942,78.446724,8860a25911fffff,17.431355,78.441867,78.447182,043533,04,0430,Link,04:59,5,20230210,customerCancelled,order cancelled before rider accepted,33,35,10,1,Ameerpet,Sri Nagar Colony
2,63e57f61c3c8103bdee23301,NaN,8860a259b9fffff,17.444122,78.387210,Hyderabad,8860a259bbfffff,17.441753,8860a259bbfffff,17.441742,78.382489,8860b196b9fffff,17.496080,78.419732,78.382538,044857,04,0445,Link,04:59,5,20230210,expired,None,57,48,10,1,Hitech City,Balanagar
3,63e57f7a446060795b9ed072,2.417,8860b5251dfffff,17.329424,78.565979,Hyderabad,8860b52ecdfffff,17.369417,8860b52ecdfffff,17.369176,78.573872,8860b5251dfffff,17.329424,78.565979,78.573624,044922,04,0445,Link,04:59,5,20230210,dropped,None,22,49,10,1,Nagole,Shamshabad
4,63e5800b446060795b9ed0be,1.249,8860a25903fffff,17.440149,78.450714,Hyderabad,8860a25939fffff,17.442627,8860a25939fffff,17.442596,78.445930,8860a25903fffff,17.440149,78.450714,78.445938,045147,04,0445,Link,04:59,5,20230210,dropped,None,47,51,10,1,Nature Cure Hostipal,Begumpet


In [ ]:
cluster_grouped_morning = df[df['hour'].isin(['08', '09', '10', '11'])].groupby(['pickup_cluster'])['order_id'].nunique().reset_index()
cluster_grouped_evening = df[df['hour'].isin(['17', '18', '19', '20', '21'])].groupby(['pickup_cluster'])['order_id'].nunique().reset_index()
cluster_grouped_morning = cluster_grouped_morning.sort_values(by = 'order_id', ascending=False)
cluster_grouped_evening = cluster_grouped_evening.sort_values(by = 'order_id', ascending=False)

top_clusters_morning = cluster_grouped_morning.head(40)
top_clusters_morning['orders_per_day'] = top_clusters_morning['order_id'].apply(lambda x: x/5)
top_clusters_morning['orders_per_hour'] = top_clusters_morning['orders_per_day'].apply(lambda x: x/4)
top_clusters_morning['orders_per_20_sec_batch'] = top_clusters_morning['orders_per_hour'].apply(lambda x: x/180)

top_clusters_evening = cluster_grouped_evening.head(40)
top_clusters_evening['orders_per_day'] = top_clusters_evening['order_id'].apply(lambda x: x/5)
top_clusters_evening['orders_per_hour'] = top_clusters_evening['orders_per_day'].apply(lambda x: x/4)
top_clusters_evening['orders_per_20_sec_batch'] = top_clusters_evening['orders_per_hour'].apply(lambda x: x/180)
top_clusters_morning

,pickup_cluster,order_id,orders_per_day,orders_per_hour,orders_per_20_sec_batch
59,Hitech City,25443,5088.6,1272.15,7.067500
1,AOC Center,7464,1492.8,373.20,2.073333
160,Shamshabad,7040,1408.0,352.00,1.955556
19,Banjara Hills,6830,1366.0,341.50,1.897222
105,Manikonda,6617,1323.4,330.85,1.838056
83,Kothaguda 2,6454,1290.8,322.70,1.792778
12,Attapur 2,6384,1276.8,319.20,1.773333
86,Kukatpally 3,6294,1258.8,314.70,1.748333
50,Golconda,6280,1256.0,314.00,1.744444
115,Mothinagar,6192,1238.4,309.60,1.720000


In [ ]:
cluster_list = top_clusters_morning['pickup_cluster'].unique().tolist()
print(cluster_list)

['Hitech City', 'AOC Center', 'Shamshabad', 'Banjara Hills', 'Manikonda', 'Kothaguda 2', 'Attapur 2', 'Kukatpally 3', 'Golconda', 'Mothinagar', 'Ameenpur', 'Miyapur', 'Mehedipatnam', 'Kothaguda', 'Ameerpet', 'Nagole', 'Begumpet', 'Kukatpally', 'Bodduppal', 'Hitech City 2', 'Dilsukhnagar', 'Saroor Nagar', 'Nallagandla', 'Banjara Hills 2', 'Kondapur', 'Quthbullapur', 'Balanagar', 'Madhapur', 'Kukatpally Phase 6', 'Marredpally', 'Alwal', 'Kachiguda', 'Miyapur 2', 'Gachibowli', 'Erragadda', 'Abids', 'Jeedimetla', 'Sri Nagar Colony', 'Nizampet', 'Kavuri Hills']


In [38]:
cluster_grouped_morning[cluster_grouped_morning['pickup_cluster'].str.contains('Saroor')]

,pickup_cluster,order_id
152,Saroor Nagar,4223
153,Saroor Nagar Lake Park,568


In [32]:
hyderabad_morning_zone1 = cluster_grouped_morning[cluster_grouped_morning['pickup_cluster'].isin(['Hitech City', 'Banjara Hills', 'Manikonda', 'Kothaguda 2', 'Kukatpally 3',
 'Mothinagar', 'Mehedipatnam', 'Kothaguda', 'Ameerpet', 'Kukatpally', 'Hitech City 2', 'Banjara Hills 2', 'Kondapur', 'Madhapur',
  'Kukatpally Phase 6', 'Gachibowli', 'Erragadda', 'Kavuri Hills', 'Jubilee Hills', 'Jubilee Hills Metro', 'Jubilee Hills Hylam Colony',
  'Jubilee Hills KBR National Park', 'Jubilee Hills Apollo Hospital', 'Botanical Garden Gachibowli', 'Telecom Nagar Gachibowli',
   'Timber Lake Colony Gachibowli', 'Wipro Gachibowli Campus', 'Bio Diversity Park Gachibowli', 'Maulana Azad Urdu University', 'Sri Ramnagar',
    'Kukatpally Phase 5 7', 'Doctors Colony Madhapur', 'Madhapur Metro', 'Sri Nagar Colony', 'Yousufguda Metro', 'SRT Nagar', 'Nature Cure Hostipal',
    'Madhura Nagar Metro', 'Sanathnagar', 'ESI Hospital Metro', 'Yousufguda', 'Durgam Cheruvu'])]
hyderabad_morning_zone1['order_id'].sum()

143090

In [33]:
((143090/5)/4)/180

39.74722222222222

In [34]:
df[(df['hour'].isin(['08', '09', '10', '11'])) & (df['pickup_cluster'].isin(['Hitech City', 'Banjara Hills', 'Manikonda', 'Kothaguda 2', 'Kukatpally 3',
 'Mothinagar', 'Mehedipatnam', 'Kothaguda', 'Ameerpet', 'Kukatpally', 'Hitech City 2', 'Banjara Hills 2', 'Kondapur', 'Madhapur',
  'Kukatpally Phase 6', 'Gachibowli', 'Erragadda', 'Kavuri Hills', 'Jubilee Hills', 'Jubilee Hills Metro', 'Jubilee Hills Hylam Colony',
  'Jubilee Hills KBR National Park', 'Jubilee Hills Apollo Hospital', 'Botanical Garden Gachibowli', 'Telecom Nagar Gachibowli',
   'Timber Lake Colony Gachibowli', 'Wipro Gachibowli Campus', 'Bio Diversity Park Gachibowli', 'Maulana Azad Urdu University', 'Sri Ramnagar',
    'Kukatpally Phase 5 7', 'Doctors Colony Madhapur', 'Madhapur Metro', 'Sri Nagar Colony', 'Yousufguda Metro', 'SRT Nagar', 'Nature Cure Hostipal',
    'Madhura Nagar Metro', 'Sanathnagar', 'ESI Hospital Metro', 'Yousufguda', 'Durgam Cheruvu']))].to_csv('hyderabad_morning_demand_zone1.csv', index = False)

In [35]:
df[df['hour'].isin(['08', '09', '10', '11'])]['order_id'].nunique()

365565

In [45]:
hyderabad_morning_zone2 = cluster_grouped_morning[cluster_grouped_morning['pickup_cluster'].isin(['Shamshabad', 'Nagole', 'Dilsukhnagar', 'Saroor Nagar',
 'Saroor Nagar Lake Park', 'Bahadurguda', 'kharmanghat', 'Attapur 2', 'Santosh Nagar West', 'Malakpet 2', 'Victoria Memorial Metro', 'shilparamam',
 'Hayath Nagar', 'L B Nagar Metro'])]
hyderabad_morning_zone2['order_id'].sum()

38561

In [46]:
((38561/5)/4)/180

10.711388888888889

In [47]:
38561 + 143090

181651

In [54]:
(38561 + 143090)/365565

0.49690479121360087

In [48]:
df['order_id'].nunique()

1291347

In [49]:
181651/1291347

0.14066784528093534

In [50]:
hyderabad_evening_zone1 = cluster_grouped_evening[cluster_grouped_evening['pickup_cluster'].isin(['Hitech City', 'Banjara Hills', 'Manikonda', 'Kothaguda 2', 'Kukatpally 3',
 'Mothinagar', 'Mehedipatnam', 'Kothaguda', 'Ameerpet', 'Kukatpally', 'Hitech City 2', 'Banjara Hills 2', 'Kondapur', 'Madhapur',
  'Kukatpally Phase 6', 'Gachibowli', 'Erragadda', 'Kavuri Hills', 'Jubilee Hills', 'Jubilee Hills Metro', 'Jubilee Hills Hylam Colony',
  'Jubilee Hills KBR National Park', 'Jubilee Hills Apollo Hospital', 'Botanical Garden Gachibowli', 'Telecom Nagar Gachibowli',
   'Timber Lake Colony Gachibowli', 'Wipro Gachibowli Campus', 'Bio Diversity Park Gachibowli', 'Maulana Azad Urdu University', 'Sri Ramnagar',
    'Kukatpally Phase 5 7', 'Doctors Colony Madhapur', 'Madhapur Metro', 'Sri Nagar Colony', 'Yousufguda Metro', 'SRT Nagar', 'Nature Cure Hostipal',
    'Madhura Nagar Metro', 'Sanathnagar', 'ESI Hospital Metro', 'Yousufguda', 'Durgam Cheruvu'])]
hyderabad_evening_zone1['order_id'].sum()

152519

In [51]:
hyderabad_evening_zone2 = cluster_grouped_evening[cluster_grouped_evening['pickup_cluster'].isin(['Shamshabad', 'Nagole', 'Dilsukhnagar', 'Saroor Nagar',
 'Saroor Nagar Lake Park', 'Bahadurguda', 'kharmanghat', 'Attapur 2', 'Santosh Nagar West', 'Malakpet 2', 'Victoria Memorial Metro', 'shilparamam',
 'Hayath Nagar', 'L B Nagar Metro'])]
hyderabad_evening_zone2['order_id'].sum()

43902

In [52]:
df[df['hour'].isin(['17', '18', '19', '20', '21'])]['order_id'].nunique()

394024

In [53]:
(152519+43902)/394024

0.498500091364993

In [55]:
(152519+43902+181651)/1291347

0.2927733599102333

In [44]:
df[(df['hour'].isin(['08', '09', '10', '11'])) & (df['pickup_cluster'].isin(['Shamshabad', 'Nagole', 'Dilsukhnagar', 'Saroor Nagar',
 'Saroor Nagar Lake Park', 'Bahadurguda', 'kharmanghat', 'Attapur 2', 'Santosh Nagar West', 'Malakpet 2', 'Victoria Memorial Metro', 'shilparamam',
 'Hayath Nagar']))].to_csv('hyderabad_morning_demand_zone2.csv', index = False)

In [37]:
143090/365565

0.39142149822876915

In [12]:
top_clusters_evening

,pickup_cluster,order_id,orders_per_day,orders_per_hour,orders_per_20_sec_batch
59,Hitech City,27576,5515.2,1378.80,7.660000
19,Banjara Hills,10533,2106.6,526.65,2.925833
112,Mehedipatnam,8105,1621.0,405.25,2.251389
60,Hitech City 2,7870,1574.0,393.50,2.186111
161,Shamshabad,7453,1490.6,372.65,2.070278
12,Attapur 2,7435,1487.0,371.75,2.065278
1,AOC Center,6426,1285.2,321.30,1.785000
38,Dilsukhnagar,6116,1223.2,305.80,1.698889
50,Golconda,6101,1220.2,305.05,1.694722
2,Abids,6059,1211.8,302.95,1.683056


In [15]:
df['weekday_name'] = df['weekday'].apply(lambda x: calendar.day_name[(x-1)])
df[df['hour'].isin(['08', '09', '10', '11'])].to_csv('hyderabad_morning_demand.csv', index = False)
df[df['hour'].isin(['17', '18', '19', '20', '21'])].to_csv('hyderabad_evening_demand.csv', index = False)

In [12]:
#Filter the df on morning and evening peak hours
df_morning = df[df['hour'].isin(['08', '09', '10', '11'])]
df_evening = df[df['hour'].isin(['17', '18', '19', '20', '21'])]

#Groupby on hex and quarter hour to get the number of orders coming in every hex
df_morning_grouped = df_morning.groupby(['yyyymmdd', 'hour', 'pickup_cluster'])['order_id'].nunique().reset_index()
df_evening_grouped = df_evening.groupby(['yyyymmdd', 'hour', 'pickup_cluster'])['order_id'].nunique().reset_index()
df_evening_grouped.head()

,yyyymmdd,hour,pickup_cluster,order_id
0,20230206,17,,58
1,20230206,17,AOC Center,297
2,20230206,17,Abids,294
3,20230206,17,Alwal,137
4,20230206,17,Alwal 2,119


In [15]:
#Query data for online captains

q_online_captains = """
select
        yyyymmdd, substr(hhmm, 1, 2) as hour, cluster,
        count(distinct captain_id) as Online_Captains
from 
(select 
        a.*, city, cluster
        
from 
        hive.datasets.supplycursory_history a 
        left join datasets.city_cluster_hex b 
        on a.location = b.hex_id and a.resolution = cast(b.resolution as varchar)
where 
        yyyymmdd >= '20230206'
        and yyyymmdd <= '20230212'
        and city in ('Hyderabad')
        and a.resolution = '8' 
        and service in ('Link')
        and substr(hhmm, 1, 2) in ('17', '18', '19', '20', '21')
)
group by 1,2,3
"""

#Load data into pandas table
df_online_captains_copy = pd.read_sql(q_online_captains, connection)

In [16]:
#Make a copy of the dataset and view it
df_online_captains = df_online_captains_copy.copy()
df_online_captains.head()

,yyyymmdd,hour,cluster,Online_Captains
0,20230211,21,Sri Nagar Colony,231
1,20230209,18,,190
2,20230210,18,Kothaguda,296
3,20230208,21,ECIL,339
4,20230209,18,Malakpet 2,193


In [17]:
#Rename cluster column
df_online_captains = df_online_captains.rename(columns = {'cluster':'pickup_cluster'})
df_online_captains.head()

,yyyymmdd,hour,pickup_cluster,Online_Captains
0,20230211,21,Sri Nagar Colony,231
1,20230209,18,,190
2,20230210,18,Kothaguda,296
3,20230208,21,ECIL,339
4,20230209,18,Malakpet 2,193


In [18]:
#Merge online captains with demand data
df_morning_grouped = df_morning_grouped.merge(df_online_captains, on = ['yyyymmdd', 'hour', 'pickup_cluster'], how = 'left')
df_evening_grouped = df_evening_grouped.merge(df_online_captains, on = ['yyyymmdd', 'hour', 'pickup_cluster'], how = 'left')
df_morning_grouped = df_morning_grouped.rename(columns = {'order_id':'num_of_orders'})
df_evening_grouped = df_evening_grouped.rename(columns = {'order_id':'num_of_orders'})
df_morning_grouped['demand_to_supply_ratio'] = round(df_morning_grouped['num_of_orders']/df_morning_grouped['Online_Captains'], 4)
df_evening_grouped['demand_to_supply_ratio'] = round(df_evening_grouped['num_of_orders']/df_evening_grouped['Online_Captains'], 4)

df_morning_grouped = df_morning_grouped.sort_values(by = 'demand_to_supply_ratio', ascending = False)
df_evening_grouped = df_evening_grouped.sort_values(by = 'demand_to_supply_ratio', ascending = False)
df_evening_grouped.head()

,yyyymmdd,hour,pickup_cluster,num_of_orders,Online_Captains,demand_to_supply_ratio
4209,20230210,19,Maulana Azad Urdu University,30,12.0,2.5000
4279,20230210,19,West Marredpally,90,55.0,1.6364
4130,20230210,19,Botanical Garden Gachibowli,225,143.0,1.5734
2227,20230208,18,Wipro Nanakramguda Circle,362,239.0,1.5146
1299,20230207,18,Wipro Nanakramguda Circle,377,251.0,1.5020


In [19]:
df_morning = df_morning.merge(df_morning_grouped, on = ['yyyymmdd', 'pickup_cluster', 'hour'], how = 'left')
df_evening = df_evening.merge(df_evening_grouped, on = ['yyyymmdd', 'pickup_cluster', 'hour'], how = 'left')
df_morning.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,quarter_hour,service_obj_service_name,time_bucket,weekday,order_status,cancel_reason,second,minute,date,count,drop_cluster,num_of_orders,yyyymmdd,hour,pickup_cluster,Online_Captains,demand_to_supply_ratio


In [20]:
#Create a column for weekdays
df_morning['weekday_name'] = df_morning['weekday'].apply(lambda x: calendar.day_name[(x-1)])
df_evening['weekday_name'] = df_evening['weekday'].apply(lambda x: calendar.day_name[(x-1)])
df_morning.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,quarter_hour,service_obj_service_name,time_bucket,weekday,order_status,cancel_reason,second,minute,date,count,drop_cluster,num_of_orders,yyyymmdd,hour,pickup_cluster,Online_Captains,demand_to_supply_ratio,weekday_name


In [21]:
num_of_hex_morning = df_morning.groupby(['pickup_cluster'])['pickup_location_hex_8'].nunique().reset_index()
num_of_hex_evening = df_evening.groupby(['pickup_cluster'])['pickup_location_hex_8'].nunique().reset_index()
num_of_hex_morning = num_of_hex_morning.rename(columns = {'pickup_location_hex_8':'num_of_hexes'})
num_of_hex_evening = num_of_hex_evening.rename(columns = {'pickup_location_hex_8':'num_of_hexes'})

df_morning = df_morning.merge(num_of_hex_evening, on = ['pickup_cluster'], how = 'left')
df_evening = df_evening.merge(num_of_hex_evening, on = ['pickup_cluster'], how = 'left')
df_morning['online_captains_per_hex'] = round(df_morning['Online_Captains']/df_morning['num_of_hexes'], 4)
df_evening['online_captains_per_hex'] = round(df_evening['Online_Captains']/df_evening['num_of_hexes'], 4)

In [22]:
df_morning_filter = df_morning[['captain_location_hex_8', 'captain_location_latitude', 'captain_location_longitude', 'pickup_location_hex_8',
 'pickup_location_latitude', 'pickup_location_longitude', 'pickup_cluster', 'hour', 'yyyymmdd', 'drop_location_hex_8', 'drop_location_latitude',
  'drop_location_longitude', 'num_of_orders', 'drop_cluster', 'Online_Captains', 'demand_to_supply_ratio', 'weekday_name', 'online_captains_per_hex']]
df_evening_filter = df_evening[['captain_location_hex_8', 'captain_location_latitude', 'captain_location_longitude', 'pickup_location_hex_8',
 'pickup_location_latitude', 'pickup_location_longitude', 'pickup_cluster', 'hour', 'yyyymmdd', 'drop_location_hex_8', 'drop_location_latitude',
  'drop_location_longitude', 'num_of_orders', 'drop_cluster', 'Online_Captains', 'demand_to_supply_ratio', 'weekday_name', 'online_captains_per_hex']]

In [23]:
df_evening_filter.to_csv('hyderabad_demand_supply_evening.csv', index = False)